# Archaeal Blast Data Handling 

## 1.1 Create Archaeal Master file

In [15]:
import os

# Directory containing blast result files
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits"

# Name of the master file
master_filename = "archaeal_master_blaster.txt"

# Path to the master file
master_filepath = os.path.join(blast_output_directory, master_filename)

# Open the master file in append mode
with open(master_filepath, "a") as master_file:
    # Iterate through each file in the blast output directory
    for filename in os.listdir(blast_output_directory):
        # Check if the file is a blast result file
        if filename.endswith("_blastp_results.txt"):
            # Get the full path of the blast result file
            blast_result_filepath = os.path.join(blast_output_directory, filename)
            # Open the blast result file and read its content
            with open(blast_result_filepath, "r") as blast_result_file:
                # Read the content of the blast result file and write it to the master file
                master_file.write(blast_result_file.read())
                # Write a newline character to separate the content of each blast result file
                master_file.write("\n")

print("All blast_results.txt files have been concatenated into the master file:", master_filepath)

All blast_results.txt files have been concatenated into the master file: /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/archaeal_master_blaster.txt


## 1.2 Print Total Hits and Accession ID's

In [29]:
from collections import defaultdict
import csv

# Path to the master file
master_filepath = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/archaeal_master_blaster.txt"

# Initialize a defaultdict to store query counts and hits
query_hits = defaultdict(list)

# Read the file and extract accession IDs and hits
try:
    with open(master_filepath, "r") as master_file:
        current_accession_id = None
        for line in master_file:
            if "|" in line:  
                current_accession_id = line.split("|")[1].strip()
            elif ">" in line:  
                if current_accession_id:
                    query_hits[current_accession_id].append(line.split(">")[1].strip())  
except FileNotFoundError:
    print(f"Error: File '{master_filepath}' not found.")
    exit(1)
except Exception as e:
    print(f"An error occurred: {e}")
    exit(1)

# Calculate total hits for each accession ID
total_hits = {accession_id: len(hits) for accession_id, hits in query_hits.items()}

# Rank and print total hits
print("Ranked total hits for each accession ID:")
for rank, (accession_id, hits) in enumerate(sorted(total_hits.items(), key=lambda x: x[1], reverse=True), start=1):
    print(f"{rank}. Accession ID: {accession_id}, Total Hits: {hits}")

# Write results to CSV file
output_csv_path = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/archaeal_master_blaster_results.csv"
try:
    with open(output_csv_path, mode='w', newline='') as csv_file:
        fieldnames = ['Accession ID', 'Hits']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        writer.writeheader()
        for accession_id, hits in query_hits.items():
            writer.writerow({'Accession ID': accession_id, 'Hits': ', '.join(hits)})
    print(f"Results saved to '{output_csv_path}'")
except Exception as e:
    print(f"An error occurred while writing to CSV: {e}")


Ranked total hits for each accession ID:
1. Accession ID: Q02388, Total Hits: 250
2. Accession ID: P11802, Total Hits: 250
3. Accession ID: P09914, Total Hits: 250
4. Accession ID: P60484, Total Hits: 250
5. Accession ID: Q00535, Total Hits: 250
6. Accession ID: P06493, Total Hits: 250
7. Accession ID: P43490, Total Hits: 250
8. Accession ID: Q9Y3Z3, Total Hits: 250
9. Accession ID: Q15303, Total Hits: 250
10. Accession ID: P19525, Total Hits: 250
11. Accession ID: P19838, Total Hits: 250
12. Accession ID: Q00534, Total Hits: 250
13. Accession ID: O95786, Total Hits: 250
14. Accession ID: O14879, Total Hits: 250
15. Accession ID: Q13325, Total Hits: 250
16. Accession ID: P56199, Total Hits: 250
17. Accession ID: Q00526, Total Hits: 250
18. Accession ID: P24941, Total Hits: 250
19. Accession ID: P01019, Total Hits: 250
20. Accession ID: P06396, Total Hits: 206
21. Accession ID: Q12965, Total Hits: 56
22. Accession ID: O43745, Total Hits: 37
23. Accession ID: O43808, Total Hits: 33
24. A

### 1.2.1 Make additions to masterfile to analyse hits in regard to protein type and name

In [2]:
import pandas as pd

# File paths
archaeal_master_blaster_filepath = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/archaeal_master_blaster_results.csv"
protein_names_filepath = "/home/osheakes/Research_Project_MMM/Fasta/Fungal_Blasts/Fungal_Hits/protein_names.csv"
output_csv_path = "/home/osheakes/Research_Project_MMM/Fasta/Fungal_Blasts/Fungal_Hits/archaeal_master_blaster_protein_results.csv"

# Read the bacterial master blaster CSV
master_blaster_df = pd.read_csv(archaeal_master_blaster_filepat)

# Read the protein names CSV
protein_names_df = pd.read_csv(protein_names_filepath)

# Merge the dataframes on 'Accession ID'
merged_df = pd.merge(master_blaster_df, protein_names_df[['Accession ID', 'Protein Names', 'Protein Type']], on='Accession ID', how='left')

# Reorder columns to place 'Protein Names' and 'Protein Type' at the start
cols = ['Protein Names', 'Protein Type'] + [col for col in bacterial_master_blaster_df.columns if col not in ['Protein Names', 'Protein Type']]
merged_df = merged_df[cols]

# Write the merged dataframe to a new CSV file
merged_df.to_csv(output_csv_path, index=False)

# Count the number of unique protein names
unique_protein_names = merged_df['Protein Names'].nunique()
print(f"Number of unique protein names: {unique_protein_names}")

# Breakdown of each protein type
protein_type_counts = merged_df['Protein Type'].value_counts()
print("Breakdown of protein types:")
print(protein_type_counts)

print(f"Merged results saved to '{output_csv_path}'")


NameError: name 'archaeal_master_blaster_filepat' is not defined

## 1.3 Filter for Queries which Produced Hits 

In [30]:
import os
import shutil

# Function to check if the given file contains the phrase "No hits found" (case-insensitive)
def contains_no_hits(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
        return "no hits found" not in content.lower()  

# Source directory containing blast result files
source_dir = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/"

# Destination directory for files with hits
dest_dir = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits"

# Create destination directory if it doesn't exist
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

# Loop through all files in the source directory
for file_name in os.listdir(source_dir):
    if file_name.endswith(".txt"):  # Check if it's a text file
        file_path = os.path.join(source_dir, file_name)
        if contains_no_hits(file_path):
            # Move the file to the destination directory if it contains hits
            shutil.move(file_path, os.path.join(dest_dir, file_name))


## 1.4 Create Initial Species CSV file from Master File

In [7]:
import os
import csv

# Source directory containing blast result files with hits
source_dir = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits"

# Output CSV file path
output_csv = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/archaeal_hits_data.csv"

# Function to extract query ID from file name
def extract_query_id(file_name):
    return file_name.replace("_blastp_results.txt", "")

# Function to parse data from blast result file
def parse_blast_results(file_path):
    hits_data = []
    with open(file_path, 'r') as f:
        query_id = extract_query_id(os.path.basename(file_path))
        hit_number = 0  # Initialize hit_number
        hit_id = None  # Initialize hit_id variable
        species_id = None  # Initialize species_id variable
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                hit_number += 1  # Increment hit_number for each new hit
                # Extract Hit ID
                hit_id = line.split(">")[1].split(" ")[0]
                # Extract species ID if present
                species_split = line.split("[")
                if len(species_split) > 1:
                    species_id = species_split[1].split("]")[0]
                else:
                    species_id = None
                # Manually add species ID for specific hit ID
                if hit_id == "MDO8640940.1":
                    species_id = "Nitrosarchaeum sp."

            elif hit_id and "Length=" in line:
                seq_length = int(line.split("=")[-1].strip())
                hits_data.append([query_id, hit_id, hit_number, seq_length, species_id])
    return hits_data

# List to store parsed data
parsed_data = []

# Loop through all files in the source directory
for file_name in os.listdir(source_dir):
    if file_name.endswith("_blastp_results.txt"): 
        file_path = os.path.join(source_dir, file_name)
        parsed_data.extend(parse_blast_results(file_path))

# Write parsed data to CSV file
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write header
    writer.writerow(['Accession ID', 'Hit ID', 'Hit Number', 'Sequence Length', 'Species ID'])
    # Write parsed data
    writer.writerows(parsed_data)

print("CSV file created successfully")


CSV file created successfully


## 1.5 Create comprehensive CSV table containing blast alignment results

In [8]:
import os
import csv

# Source directory containing blast result files with hits
source_dir = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits"

# Output CSV file path
output_csv = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/comp_archaeal_data.csv"

# Function to extract query ID from file name
def extract_query_id(file_name):
    return file_name.replace("_blastp_results.txt", "")

# Parse out required data from blast reaults
def parse_blast_results(file_path):
    hits_data = []
    with open(file_path, 'r') as f:
        query_id = extract_query_id(os.path.basename(file_path))
        hit_number = 0 
        hit_id = None 
        species_id = None 
        seq_length = None  
        evalue = None 
        score = None  
        identities = None  
        identities_percent = None
        positives = None 
        positives_percent = None  
        gaps = None  
        gaps_percent = None  
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                hit_number += 1 
                hit_id = line.split(">")[1].split(" ")[0]
                species_split = line.split("[")
                if len(species_split) > 1:
                    species_id = species_split[1].split("]")[0]
                else:
                    species_id = None
                # Manually add species ID for specific hit ID
                if hit_id == "MDO8640940.1":
                    species_id = "Nitrosarchaeum sp."

            elif hit_id and "Length=" in line:
                seq_length = int(line.split("=")[-1].strip())
            elif hit_id and "Expect =" in line:
                evalue = line.split("Expect = ")[1].split(",")[0].strip()
                score = line.split("Score = ")[1].split(" bits")[0].strip()
            elif hit_id and "Identities =" in line:
                identities_str = line.split("Identities = ")[1].split(",")[0].strip()
                identities = identities_str.split(" ")[0]
                identities_fraction = identities.split("/")
                identities_percent = (int(identities_fraction[0]) / int(identities_fraction[1])) * 100
                positives_str = line.split("Positives = ")[1].split(",")[0].strip()
                positives = positives_str.split(" ")[0]
                positives_fraction = positives.split("/")
                positives_percent = (int(positives_fraction[0]) / int(positives_fraction[1])) * 100
                gaps_str = line.split("Gaps = ")[1].split(" ")[0].strip()
                gaps = gaps_str
                gaps_fraction = gaps.split("/")
                gaps_percent = (int(gaps_fraction[0]) / int(gaps_fraction[1])) * 100
                # Append all values to hits_data
                hits_data.append([
                    query_id, hit_id, hit_number, seq_length, species_id, evalue, score, 
                    identities, f"{identities_percent:.2f}%", positives, f"{positives_percent:.2f}%", gaps, f"{gaps_percent:.2f}%"
                ])
    return hits_data

# List to store parsed data
parsed_data = []

# Loop through all files in the source directory
for file_name in os.listdir(source_dir):
    if file_name.endswith("_blastp_results.txt"): 
        file_path = os.path.join(source_dir, file_name)
        parsed_data.extend(parse_blast_results(file_path))

# Write parsed data to CSV file
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write header
    writer.writerow([
        'Accession ID', 'Hit ID', 'Hit Number', 'Sequence Length', 'Species ID', 'Evalue', 'Score', 
        'Identities', 'Identities (%)', 'Positives', 'Positives (%)', 'Gaps', 'Gaps (%)'
    ])
    # Write parsed data
    writer.writerows(parsed_data)

print("CSV file created successfully")


CSV file created successfully


In [9]:
import os
import csv

# Source directory containing blast result files with hits
source_dir = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits"

# Output CSV file path
output_csv = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/comp_archaeal_data.csv"

# Function to extract query ID from file name
def extract_query_id(file_name):
    return file_name.replace("_blastp_results.txt", "")

# Parse out required data from blast reaults
def parse_blast_results(file_path):
    hits_data = []
    with open(file_path, 'r') as f:
        query_id = extract_query_id(os.path.basename(file_path))
        hit_number = 0 
        hit_id = None 
        species_id = None 
        seq_length = None  
        evalue = None 
        score = None  
        identities = None  
        identities_percent = None
        positives = None 
        positives_percent = None  
        gaps = None  
        gaps_percent = None  
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                hit_number += 1 
                hit_id = line.split(">")[1].split(" ")[0]
                species_split = line.split("[")
                if len(species_split) > 1:
                    species_id = species_split[1].split("]")[0]
                else:
                    species_id = None
               

            elif hit_id and "Length=" in line:
                seq_length = int(line.split("=")[-1].strip())
            elif hit_id and "Expect =" in line:
                evalue = line.split("Expect = ")[1].split(",")[0].strip()
                score = line.split("Score = ")[1].split(" bits")[0].strip()
            elif hit_id and "Identities =" in line:
                identities_str = line.split("Identities = ")[1].split(",")[0].strip()
                identities = identities_str.split(" ")[0]
                identities_fraction = identities.split("/")
                identities_percent = (int(identities_fraction[0]) / int(identities_fraction[1])) * 100
                positives_str = line.split("Positives = ")[1].split(",")[0].strip()
                positives = positives_str.split(" ")[0]
                positives_fraction = positives.split("/")
                positives_percent = (int(positives_fraction[0]) / int(positives_fraction[1])) * 100
                gaps_str = line.split("Gaps = ")[1].split(" ")[0].strip()
                gaps = gaps_str
                gaps_fraction = gaps.split("/")
                gaps_percent = (int(gaps_fraction[0]) / int(gaps_fraction[1])) * 100
                # Append all values to hits_data
                hits_data.append([
                    query_id, hit_id, hit_number, seq_length, species_id, evalue, score, 
                    identities, f"{identities_percent:.2f}%", positives, f"{positives_percent:.2f}%", gaps, f"{gaps_percent:.2f}%"
                ])
    return hits_data

# List to store parsed data
parsed_data = []

# Loop through all files in the source directory
for file_name in os.listdir(source_dir):
    if file_name.endswith("_blastp_results.txt"): 
        file_path = os.path.join(source_dir, file_name)
        parsed_data.extend(parse_blast_results(file_path))

# Write parsed data to CSV file
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write header
    writer.writerow([
        'Accession ID', 'Hit ID', 'Hit Number', 'Sequence Length', 'Species ID', 'Evalue', 'Score', 
        'Identities', 'Identities (%)', 'Positives', 'Positives (%)', 'Gaps', 'Gaps (%)'
    ])
    # Write parsed data
    writer.writerows(parsed_data)

print("CSV file created successfully")


CSV file created successfully


### Combine Hits Data from Viruses, Bacteria and Archaea

In [2]:
import pandas as pd
import re

# Define the file paths
bacterial_blast_file = "/home/osheakes/Research_Project_MMM/Fasta/Bacterial_Blasts/bacterial_master_blaster_results.csv"
viral_csv_file = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/processed_viral_seq/viral_hits.csv"
output_csv_file = "/home/osheakes/Research_Project_MMM/Fasta/Bacterial_Blasts/Bacterial_Hits/bac_vs_vir_hits.csv"

# Function to clean accession IDs
def clean_accession_id(accession_id):
    return re.sub(r'\s+', '', str(accession_id)).upper()

# Function to extract accession IDs from a CSV file
def extract_accession_ids_from_csv(csv_file):
    df = pd.read_csv(csv_file, encoding='utf-8')
    return sorted(set(df['Accession ID'].apply(clean_accession_id)))

# Extract and sort accession IDs from the bacterial and viral CSV files
bacterial_accession_ids = extract_accession_ids_from_csv(bacterial_blast_file)
viral_accession_ids = extract_accession_ids_from_csv(viral_csv_file)

# Create a DataFrame with two columns: "viral hits" and "bacterial hits"
max_length = max(len(bacterial_accession_ids), len(viral_accession_ids))
bacterial_accession_ids += [""] * (max_length - len(bacterial_accession_ids)) 
viral_accession_ids += [""] * (max_length - len(viral_accession_ids))  

data = {
    'viral hits': viral_accession_ids,
    'bacterial hits': bacterial_accession_ids
}

df = pd.DataFrame(data)

# Save the DataFrame to a new CSV file
df.to_csv(output_csv_file, index=False)

print(f"CSV file with viral and bacterial hits has been saved to {output_csv_file}")

CSV file with viral and bacterial hits has been saved to /home/osheakes/Research_Project_MMM/Fasta/Bacterial_Blasts/Bacterial_Hits/bac_vs_vir_hits.csv


### Compare Hits From Viral and Bacterial Search

In [ ]:
import os
import pandas as pd
import shutil

# Define file paths and directories
csv_file_path = "/home/osheakes/Research_Project_MMM/Fasta/Bacterial_Blasts/Bacterial_Hits/bac_vs_vir_hits.csv"
source_directory = "/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/processed_viral_seq"
destination_directory = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_archaeal_seq"
# Ensure the destination directory exists
os.makedirs(destination_directory, exist_ok=True)

# Read the CSV file to get the Accession IDs from the "bacterial hits" column
df = pd.read_csv(csv_file_path)
bacterial_hits = df['bacterial hits'].dropna().unique()

# Function to check if a file has a matching Accession ID
def has_matching_accession_id(file_name, accession_ids):
    # Extract the Accession ID from the file name
    accession_id = os.path.splitext(file_name)[0]
    return accession_id in accession_ids

# Counter to keep track of the number of files copied
num_files_copied = 0

# Scan the source directory for matching fasta files and copy them
for file_name in os.listdir(source_directory):
    if file_name.endswith(".fasta") and has_matching_accession_id(file_name, bacterial_hits):
        source_file_path = os.path.join(source_directory, file_name)
        destination_file_path = os.path.join(destination_directory, file_name)
        shutil.copy2(source_file_path, destination_file_path)
        num_files_copied += 1
        print(f"Copied: {file_name}")

print(f"Total files copied: {num_files_copied}")


#  

# 3.0 Run Blast on Processed Sequences

### 3.1 Create working Directory

In [18]:
!mkdir /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs

### 3.2 Pull Fasta's from Archaeal processed sequences Text file

In [26]:
import os

# Input file path
file_path = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_archaeal_blasts.txt"
# Output directory path
output_dir = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Function to extract accession ID from header line
def extract_accession_id(header):
    parts = header.split('|')
    if len(parts) > 1:
        return parts[1].strip()
    return None

# Read the input file and process
with open(file_path, 'r') as infile:
    current_file = None
    current_file_path = None
    
    for line in infile:
        if line.startswith('>'):
            # Close the previous file if it was open
            if current_file:
                current_file.close()
            
            # Extract accession ID and create a new file
            accession_id = extract_accession_id(line)
            if accession_id:
                current_file_path = os.path.join(output_dir, f"{accession_id}.fasta")
                current_file = open(current_file_path, 'w')
        
        # Write the current line to the current file
        if current_file:
            current_file.write(line)
    
    # Close the last file if it was open
    if current_file:
        current_file.close()

print("FASTA files have been successfully created.")


FASTA files have been successfully created.


### 3.3 Run Blast on Archaeal Samples

In [23]:
!mkdir  /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results

In [ ]:
import subprocess
import os
import time
from datetime import datetime, timezone

# Path to the directory containing fasta files
fasta_directory = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs"

# Directory to store blast results
blast_output_directory =  "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results"

# Create the blast output directory if it doesn't exist
os.makedirs(blast_output_directory, exist_ok=True)

# Record the start time of the entire process in GMT
start_time = datetime.now(timezone.utc)
print(f"Process started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')} GMT")

# Iterate through each fasta file in the directory
for filename in os.listdir(fasta_directory):
    if filename.endswith(".fasta"):
        fasta_file = os.path.join(fasta_directory, filename)
        output_file = os.path.join(blast_output_directory, f"{os.path.splitext(filename)[0]}_blastp_results.txt")
        
        # Skip if output file already exists
        if os.path.exists(output_file):
            print(f"Skipping {filename} as output file already exists.")
            continue
        
        # Construct the blastp command
        blastp_command = [
            "blastp",
            "-db",
            "nr",
            "-query",
            fasta_file,
            "-out",
            output_file,
            "-evalue",
            "0.06",
            "-remote",
            "-entrez_query",
            "archaea[orgn]"
        ]
        
        # Record start time for this file
        file_start_time = time.time()
        
        # Execute the blastp command
        print(f"Running blast search for {filename}...")
        try:
            result = subprocess.run(blastp_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
            result.check_returncode()  
            print(f"Blast search for {filename} completed successfully.")
        except subprocess.CalledProcessError as e:
            print(f"Blast search for {filename} failed with return code {e.returncode}.")
            print(f"Error: {e.stderr}")
        
        # Calculate elapsed time for this file
        file_elapsed_time = time.time() - file_start_time
        
        # Print filename and elapsed time
        print(f"Blast search for {filename} completed in {file_elapsed_time:.2f} seconds.")
        
        # Check if 30 minutes have elapsed since the start of the process
        if (datetime.now(timezone.utc) - start_time).seconds >= 1800:
            print(f"Process has been running for 30 minutes. Current time: {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S')} GMT")
            start_time = datetime.now(timezone.utc)  

print("Blastp searches completed for all fasta files.")

## 4.0 Collate Processed Results

### 4.1 Create Masterfile for Archaeal results

In [29]:
import os

# Directory containing blast result files
blast_output_directory = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results"

# Name of the master file
master_filename = "processed_archaeal_master_blaster.txt"

# Path to the master file
master_filepath = os.path.join(blast_output_directory, master_filename)

# Open the master file in append mode
with open(master_filepath, "a") as master_file:
    # Iterate through each file in the blast output directory
    for filename in os.listdir(blast_output_directory):
        # Check if the file is a blast result file
        if filename.endswith("_blastp_results.txt"):
            # Get the full path of the blast result file
            blast_result_filepath = os.path.join(blast_output_directory, filename)
            # Open the blast result file and read its content
            with open(blast_result_filepath, "r") as blast_result_file:
                # Read the content of the blast result file and write it to the master file
                master_file.write(blast_result_file.read())
                # Write a newline character to separate the content of each blast result file
                master_file.write("\n")

print("All blast_results.txt files have been concatenated into the master file:", master_filepath)

All blast_results.txt files have been concatenated into the master file: /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/processed_archaeal_master_blaster.txt


### 4.2 Count hits and Create csv

In [43]:
from collections import defaultdict
import csv

# Path to the master file
master_filepath = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/processed_archaeal_master_blaster.txt"

# Initialize a defaultdict to store query counts and hits
query_hits = defaultdict(list)

# Read the file and extract query IDs and hits
try:
    with open(master_filepath, "r") as master_file:
        current_query_id = None
        for line in master_file:
            if "|" in line:  
                current_query_id = line.split("|")[1].strip()
            elif ">" in line:  
                if current_query_id:
                    query_hits[current_query_id].append(line.split(">")[1].strip())  
except FileNotFoundError:
    print(f"Error: File '{master_filepath}' not found.")
    exit(1)
except Exception as e:
    print(f"An error occurred: {e}")
    exit(1)

# Calculate total hits for each accession ID
total_hits = {query_id: len(hits) for query_id, hits in query_hits.items()}

# Rank and print total hits
print("Ranked total hits for each accession ID:")
for rank, (query_id, hits) in enumerate(sorted(total_hits.items(), key=lambda x: x[1], reverse=True), start=1):
    print(f"{rank}. Accession ID: {query_id}, Total Hits: {hits}")

# Write results to CSV file
output_csv_path = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/processed_archaeal_master_blaster.csv"
try:
    with open(output_csv_path, mode='w', newline='') as csv_file:
        fieldnames = ['Query ID', 'Hits']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        writer.writeheader()
        for query_id, hits in query_hits.items():
            writer.writerow({'Query ID': query_id, 'Hits': ', '.join(hits)})
    print(f"Results saved to '{output_csv_path}'")
except Exception as e:
    print(f"An error occurred while writing to CSV: {e}")


Ranked total hits for each accession ID:
1. Accession ID: Q02388, Total Hits: 250
2. Accession ID: P11802, Total Hits: 250
3. Accession ID: P09914, Total Hits: 250
4. Accession ID: P60484, Total Hits: 250
5. Accession ID: Q00535, Total Hits: 250
6. Accession ID: P06493, Total Hits: 250
7. Accession ID: Q9Y3Z3, Total Hits: 250
8. Accession ID: Q15303, Total Hits: 250
9. Accession ID: P19525, Total Hits: 250
10. Accession ID: P19838, Total Hits: 250
11. Accession ID: Q00534, Total Hits: 250
12. Accession ID: O95786, Total Hits: 250
13. Accession ID: P56199, Total Hits: 250
14. Accession ID: Q00526, Total Hits: 250
15. Accession ID: P24941, Total Hits: 250
16. Accession ID: P06396, Total Hits: 206
17. Accession ID: Q12965, Total Hits: 53
18. Accession ID: O43745, Total Hits: 35
19. Accession ID: O43808, Total Hits: 31
20. Accession ID: Q99653, Total Hits: 21
21. Accession ID: P55265, Total Hits: 19
22. Accession ID: P20592, Total Hits: 10
23. Accession ID: P20591, Total Hits: 10
24. Acces

### 4.3 Create comprehensive processed csv

In [77]:
import os
import csv

# Source directory containing blast result files with hits
source_dir = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/"

# Output CSV file path
output_csv = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/comp_archaeal_processed_results.csv"

# Function to extract query ID from file name
def extract_query_id(file_name):
    return file_name.replace("_blastp_results.txt", "")

# Parse out required data from blast results
def parse_blast_results(file_path):
    hits_data = []
    with open(file_path, 'r') as f:
        accession_id = extract_query_id(os.path.basename(file_path))
        hit_number = 0 
        hit_id = None 
        species_id = None 
        seq_length = None  
        evalue = None 
        score = None  
        identities = None  
        positives = None  
        gaps = None  
        query_definition = None  
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                hit_number += 1 
                hit_id = line.split(">")[1].split(" ")[0]
                species_split = line.split("[")
                if len(species_split) > 1:
                    species_id = species_split[1].split("]")[0]
                else:
                    species_id = None
                query_definition = line  # Assuming entire line is query definition

            elif hit_id and "Length=" in line:
                seq_length = int(line.split("=")[-1].strip())
            elif hit_id and "Expect =" in line:
                evalue = line.split("Expect = ")[1].split(",")[0].strip()
                score = line.split("Score = ")[1].split(" bits")[0].strip()
            elif hit_id and "Identities =" in line:
                identities_str = line.split("Identities = ")[1].split(",")[0].strip()
                identities = round(float(identities_str.split(" ")[0].split("/")[0]) / float(identities_str.split(" ")[0].split("/")[1]) * 100, 2)
                positives_str = line.split("Positives = ")[1].split(",")[0].strip()
                positives = positives_str.split(" ")[0]
                gaps_str = line.split("Gaps = ")[1].split(" ")[0].strip()
                gaps = gaps_str
                hits_data.append([
                    os.path.basename(file_path), accession_id, hit_number, hit_id, seq_length, species_id, 
                    query_definition, "N/A", score, evalue, identities, positives, gaps, "Query", "Subject", "Archaea"
                ])
    return hits_data

# List to store parsed data
parsed_data = []

# List to store excluded file names
excluded_files = []

# Loop through all files in the source directory
for file_name in os.listdir(source_dir):
    if file_name.endswith("_blastp_results.txt"): 
        file_path = os.path.join(source_dir, file_name)
        parsed_data.extend(parse_blast_results(file_path))
    else:
        excluded_files.append(file_name)  

# Write parsed data to CSV file
with open(output_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write header
    writer.writerow([
        "File Name", "Accession ID", "Hit Number", "Hit ID", "Hit Length", "Species", 
        "Query Definition", "Hsp Bit Score", "Hsp Score", "Hsp E-value", 
        "Identity", "Positives", "Gaps", "Hsp Qseq", "Hsp Hseq", "Database"
    ])
    # Write parsed data
    writer.writerows(parsed_data)

print(f"CSV file created successfully with {len(parsed_data)} entries.")

# Print excluded file names
if excluded_files:
    print("\nExcluded files:")
    for file_name in excluded_files:
        print(file_name)
else:
    print("\nNo files were excluded.")

CSV file created successfully with 10257 entries.

Excluded files:
processed_archaeal_master_blaster.txt
.ipynb_checkpoints
comp_archaeal_processed_results.csv
processed_archaeal_master_blaster.csv
archaeal_master_blaster.txt
final_archaeal_processed_results.csv


### 4.4 Add Protein Names

In [5]:
import csv

# Input and output file paths
input_file = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/protein_names.txt"
output_file = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/protein_names.csv"

# Initialize a dictionary to track seen combinations of protein type and accession ID
seen_combinations = {}

# Process the lines to split data and prepare for CSV writing
data = []
with open(input_file, 'r', encoding='utf-8') as txt_file:
    try:
        reader = csv.reader(txt_file, delimiter='\t')
        for line_data in reader:
            # Skip empty lines
            if not line_data or all(cell.strip() == '' for cell in line_data):
                continue
            
            # Ensure each line has enough cells to contain at least Protein Type and Accession ID
            if len(line_data) >= 2:
                protein_type = line_data[0].strip()
                accession_id = line_data[1].strip()
                if accession_id: 
                   
                    key = (protein_type, accession_id)
                    if key not in seen_combinations:
                        seen_combinations[key] = True
                        data.append(line_data)
    except UnicodeDecodeError:
        print(f"Error decoding line in file: {input_file}")
        

# Write the processed data to CSV file
with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows(data)

print(f"Converted {input_file} to CSV format at {output_file}")

Converted /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/protein_names.txt to CSV format at /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/protein_names.csv


In [14]:
import csv

# Input file paths
comp_archaeal_file = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/comp_archaeal_processed_results.csv"
protein_names_file = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/protein_names.csv"
output_file = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/final_archaeal_processed_results.csv"

# Step 1: Load data from protein_names.csv into a dictionary
accession_info = {}
with open(protein_names_file, 'r', encoding='utf-8') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        accession_id = row['Accession ID'].strip()
        protein_type = row['Protein Type'].strip()
        protein_names = row['Protein Names'].strip()
        accession_info[accession_id] = {
            'ProteinType': protein_type,
            'ProteinNames': protein_names
        }

# Step 2: Merge data with comp_archaeal_processed_results.csv and add new columns
merged_data = []
with open(comp_archaeal_file, 'r', encoding='utf-8') as csv_file:
    reader = csv.reader(csv_file)
    header = next(reader)
    header.insert(1, 'Protein Type')
    header.insert(2, 'Protein Names')
    merged_data.append(header)

    for row in reader:
        accession_id = row[1].strip()

        if accession_id in accession_info:
            protein_type = accession_info[accession_id]['ProteinType']
            protein_names = accession_info[accession_id]['ProteinNames']
        else:
            protein_type = ''
            protein_names = ''

        row.insert(1, protein_type)
        row.insert(2, protein_names)
        merged_data.append(row)

# Step 3: Write merged data to final_archaeal_processed_results.csv
with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerows(merged_data)

print(f"Merged data from {comp_archaeal_file} and {protein_names_file} into {output_file}")


Merged data from /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/comp_archaeal_processed_results.csv and /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/protein_names.csv into /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/final_archaeal_processed_results.csv


In [16]:
import csv

# Input file paths
comp_archaeal_file = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/comp_archaeal_processed_results.csv"
protein_names_file = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/protein_names.csv"
output_file = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/final_archaeal_processed_results.csv"


# Function to print headers of a CSV file
def print_csv_headers(file_path):
    with open(file_path, 'r', encoding='utf-8') as csv_file:
        # Determine the delimiter (tab or comma) by checking the first line
        first_line = csv_file.readline()
        delimiter = '\t' if '\t' in first_line else ','
        
        # Reset file pointer to beginning
        csv_file.seek(0)
        
        reader = csv.reader(csv_file, delimiter=delimiter)
        header = next(reader)
        print(f"Headers for {file_path}:")
        print(header)
        print()

# Print headers for both files
print_csv_headers(comp_archaeal_file)
print_csv_headers(protein_names_file)
print_csv_headers(output_file)

Headers for /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/comp_archaeal_processed_results.csv:
['File Name', 'Accession ID', 'Hit Number', 'Hit ID', 'Hit Length', 'Species', 'Query Definition', 'Hsp Bit Score', 'Hsp Score', 'Hsp E-value', 'Identity', 'Positives', 'Gaps', 'Hsp Qseq', 'Hsp Hseq', 'Database']

Headers for /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/protein_names.csv:
['Protein Type', 'Protein Names', 'Accession ID']

Headers for /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/final_archaeal_processed_results.csv:
['File Name', 'Protein Type', 'Protein Names', 'Accession ID', 'Hit Number', 'Hit ID', 'Hit Length', 'Species', 'Query Definition', 'Hsp Bit Score', 'Hsp Score', 'Hsp E-value', 'Identity', 'Positives', 'Gaps', 'Hsp Qseq', 'Hsp Hseq', 'Database']



### 4.5 Reorder Columns

In [18]:
import csv

# Input and output file paths
input_file = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/final_archaeal_processed_results.csv"
output_file = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/reordered_comp_archaeal_processed_results.csv"

# Define the desired column order
desired_order = ['Protein Type', 'Protein Names', 'File Name', 'Accession ID', 'Hit Number', 'Hit ID', 'Hit Length', 'Species', 'Query Definition', 'Hsp Bit Score', 'Hsp Score', 'Hsp E-value', 'Identity', 'Positives', 'Gaps', 'Hsp Qseq', 'Hsp Hseq', 'Database']

# Read the original CSV file and reorder columns
with open(input_file, 'r', newline='', encoding='utf-8') as csv_file:
    reader = csv.reader(csv_file)
    header = next(reader)
    
    # Create a dictionary to map original indices to desired indices
    header_mapping = {header.index(col): idx for idx, col in enumerate(desired_order)}
    reordered_header = [header[idx] for idx in sorted(header_mapping.keys(), key=lambda x: header_mapping[x])]
    
    # Read and reorder rows
    rows = []
    for row in reader:
        reordered_row = [row[idx] for idx in sorted(header_mapping.keys(), key=lambda x: header_mapping[x])]
        rows.append(reordered_row)

# Write reordered data to a new CSV file
with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(reordered_header)
    writer.writerows(rows)

print(f"Reordered columns in {input_file} and saved to {output_file}")


Reordered columns in /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/final_archaeal_processed_results.csv and saved to /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/reordered_comp_archaeal_processed_results.csv


## 5.0 Extract some key metrics from the Processed Search

### 5.1 Print Queries which Produced hits

In [1]:
import csv

# Path to the CSV file
csv_file_path = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/reordered_comp_archaeal_processed_results.csv"
# Initialize a set to store unique Accession IDs
unique_accession_ids = set()

try:
    # Open and read the CSV file
    with open(csv_file_path, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        
        # Loop through each row in the CSV file
        for row in csv_reader:
            # Extract the Accession ID and add it to the set
            accession_id = row['Accession ID']  # Replace 'Accession ID' with the actual column name if different
            unique_accession_ids.add(accession_id)
    
    # Print each unique Accession ID
    for accession_id in unique_accession_ids:
        print(accession_id)
    
    # Print the total number of unique Accession IDs
    print(f"Total number of unique Accession IDs: {len(unique_accession_ids)}")

except FileNotFoundError:
    print(f"Error: File '{csv_file_path}' not found.")
except KeyError:
    print("Error: 'Accession ID' column not found in the CSV file. Please check the column name.")
except Exception as e:
    print(f"An error occurred: {e}")

P60484
Q12965
P19883
P02533
P56199
Q02388
P11802
P42226
P55265
P13164
O43745
P20591
Q14765
Q99653
P40763
P19525
P24941
Q9BTW9
Q00534
P38398
Q00526
Q9Y5Y5
P06493
P09914
O43808
P23142
P06396
Q15303
P15502
P19838
Q9Y3Z3
P52630
Q96BS2
P04637
O95786
P20592
O00910
P42224
Q92968
Q00535
Total number of unique Accession IDs: 40


### 5.2 Print the number of input files

In [2]:
import os

# Path to the directory
directory_path = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/"

# Initialize a counter for FASTA files
fasta_file_count = 0

# Iterate through the files in the directory
for file_name in os.listdir(directory_path):
    if file_name.endswith(".fasta") or file_name.endswith(".fa"):  
        fasta_file_count += 1

# Print the total number of FASTA files
print(f"Total number of FASTA files: {fasta_file_count}")

Total number of FASTA files: 52


# 

## 6.0 Perform species analysis

### 6.1 Create directory

In [3]:
!mkdir  "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/species"

### 6.2 Create Unique list of species

In [ ]:
! awk -F "," '{ print $8 }' /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/reordered_comp_archaeal_processed_results.csv | sort | uniq > /home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/species/unique_archaea.txt

### 6.3 Run Taxonkit on Unique Species

In [5]:
import subprocess

# Path to the unique-species.txt file
file_path = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/species/unique_archaea.txt"
# Path to the taxonkit binary
taxonkit_path = '/home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/processed_viral_seq/species/taxonkit'

# Read the species names from the file
with open(file_path, 'r') as file:
    species_names = [line.strip() for line in file.readlines()]

# Define a function to apply the command to each species name
def process_species_name(species_name):
    # Construct the command
    cmd = f'echo "{species_name}" | {taxonkit_path} name2taxid | {taxonkit_path} reformat -I 2'
    # Run the command
    result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    # Debug: Print command and output
    print(f"Command: {cmd}")
    print(f"Stdout: {result.stdout.strip()}")
    print(f"Stderr: {result.stderr.strip()}")
    # Return the output
    return result.stdout.strip()

# Process each species name and store the results
results = {species: process_species_name(species) for species in species_names}

# Print the results
for species, output in results.items():
    print(f'{species}:\n{output}\n')

# Optionally, save the results to a new file
output_file_path = "/home/osheakes/Research_Project_MMM/Fasta/Archaeal_Blasts/Archaeal_Hits/processed_seqs/processed_archaeal_results/species/processed_species.txt"
with open(output_file_path, 'w') as output_file:
    for species, output in results.items():
        output_file.write(f'{species}:\n{output}\n\n')

Command: echo "" | /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/processed_viral_seq/species/taxonkit name2taxid | /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/processed_viral_seq/species/taxonkit reformat -I 2
Stdout: 
Stderr: 
Command: echo "Acidianus brierleyi" | /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/processed_viral_seq/species/taxonkit name2taxid | /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/processed_viral_seq/species/taxonkit reformat -I 2
Stdout: Acidianus brierleyi	41673	Archaea;Thermoproteota;Thermoprotei;Sulfolobales;Sulfolobaceae;Acidianus;Acidianus brierleyi
Stderr: 
Command: echo "Acidianus infernus" | /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/processed_viral_seq/species/taxonkit name2taxid | /home/osheakes/Research_Project_MMM/Fasta/Viral_Blasts/processed_viral_seq/species/taxonkit reformat -I 2
Stdout: Acidianus infernus	12915	Archaea;Thermoproteota;Thermoprotei;Sulfolobales;Sulfolobaceae;Acidianus;Acidianus inf